In [1]:
# import libraries
from bs4 import BeautifulSoup
import urllib.request
import csv
import pandas as pd
import numpy as np
import math

In [2]:
url = 'https://www.property.hk/eng/tran.php?dt=&bldg=&prop=&saleType=3&loc=&page='
max_page = 19

In [3]:
#create and write headers to a list
rows = []
rows.append(['Input Date','Address','Floor','Unit','Size(ft²)','Price',\
             'Price/ft²','District','Street','Building','Usage','OP Date','Facing','Layout',])

In [4]:
for p_num in range(0,max_page):
    url = url+str(p_num+1)
    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page, 'html.parser')
    table = soup.find('table',attrs={'class':'table table-hover hidden-xs'})
    targets = table.find_all('tr')

    #loop over the resualts 
    for result in targets:
        data = result.find_all('td')

        #skip value 0
        if len(data) == 0 or len(data)==1:
            continue

        #address = data[2].getText()
        input_date = data[1].getText()
        floor = data[3].getText()
        unit = data[4].getText()
        size = data[5].getText()
        total_price = data[6].getText()
        price_per_sqft = data[7].getText()
        detail = data[8]

        #extract the link of 'detail' and parse the content
        link = str('https://www.property.hk')+detail.find('a').get('href')
        page_2 = urllib.request.urlopen(link)
        soup_2 = BeautifulSoup(page_2, 'html.parser')

        #find the location of the relevant information
        data_2 = soup_2.find_all('div',attrs={'class':'col-xs-9'})

        #skip if there is no result
        if len(data_2) == 0:
            continue

        #save the information in different variables
        address = data_2[6].getText()
        district = data_2[4].getText()
        usage = data_2[5].getText()
        op_date = data_2[8].getText()
        facing = data_2[13].getText()
        layout = data_2[14].getText()

        #get the building and street information
        if len(address.split(','))<2:
            building = ''
            street = ''
        elif len(address.split(','))==2:
            building = ''
            street = address.split(',')[-1]
        else:
            building = address.split(',')[2]
            street = address.split(',')[-1]

        total_price = float(total_price)*1000000

        if price_per_sqft == '':
            price_per_sqft = '' 
        else:
            price_per_sqft = float(price_per_sqft)

        if size == '':
            size = ''
        else:
            price_per_sqft = float(price_per_sqft)   

        rows.append([input_date, address, floor, unit, size, total_price,\
                     price_per_sqft, district, street, building,usage,op_date,facing,layout])

In [5]:
#sample of the output
print("Address:",address)
print('----')
print('Floor:',floor)
print('----')
print('Unit:',unit)
print('----')
print('Size:',size)
print('----')
print('Total listed price:',total_price)
print('----')
print('Price per squarefoot:',price_per_sqft)
print('----')
print('district:',district)
print('----')
print('usage:',usage)
print('----')
print('op_date:',op_date)
print('----')
print('facing:',facing)
print('----')
print('layout:',layout)

Address: UNIT 2, FLOOR 21, TAK FUNG IND CTR BLK 01, TEXACO RD 166-176
----
Floor: 21
----
Unit: 2
----
Size: 520
----
Total listed price: 3150000.0
----
Price per squarefoot: 6058.0
----
district: Tsuen Wan
----
usage: Industrial
----
op_date: 1993
----
facing:  
----
layout:  


In [6]:
print(len(rows))

381


In [7]:
#open csv and write rows to the csv file
with open('property_transaction.csv','w', newline='') as f_output:
    csv_output = csv.writer(f_output)
    csv_output.writerows(rows)

In [8]:
data = pd.read_csv('property_transaction.csv')
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000
data.head(5)

,Input Date,Address,Floor,Unit,Size(ft²),Price,Price/ft²,District,Street,Building,Usage,OP Date,Facing,Layout
0,2021-06-25,"FLOOR 5, CANTON RD 1037",5,NaN,NaN,20000000.0,NaN,Mong Kok,CANTON RD 1037,NaN,Residential,,,
1,2021-06-25,"FLOOR 4, CANTON RD 1037",4,NaN,NaN,20000000.0,NaN,Mong Kok,CANTON RD 1037,NaN,Residential,,,
2,2021-06-25,"FLOOR 3, CANTON RD 1037",3,NaN,NaN,20000000.0,NaN,Mong Kok,CANTON RD 1037,NaN,Residential,,,
3,2021-06-25,"FLOOR 2, CANTON RD 1037",2,NaN,NaN,20000000.0,NaN,Mong Kok,CANTON RD 1037,NaN,Residential,,,
4,2021-06-25,"UNIT N8, FLOOR 5, W LUXE, ON YIU ST 5",5,N8,313.0,5800000.0,18530.0,Shatin,ON YIU ST 5,W LUXE,Office,2020,東北,LAV


In [9]:
#replace the '\xa0' values with ''

data['Facing'] = data['Facing'].replace(u'\xa0', u'')
data['Layout'] = data['Layout'].replace(u'\xa0', u'')
data['OP Date'] = data['OP Date'].replace(u'\xa0', u'')

In [10]:
#change the Chinese values in the Facing to English
f_dict = {
    '東':'East',
    '南':'South',
    '西':'West',
    '北':'North',
    '東南':'Southeastern',
    '東北':'Northeastern',
    '西北':'Northwestern',
    '西南':'Southwestern',
    '':'Unknown'
}

data = data.replace({'Facing':f_dict})

In [11]:
#all unique values in the Layout columns (luckily there are not too many)
data['Layout'].unique()

array(['', 'LAV', '3房(1套)2廳1工', '2房2廳,開放廚', '2房1廳', '3房(1套)2廳', '2房2廳',
       '1房2廳', '1房2廳,開放廚', '3房2廳'], dtype=object)

In [12]:
#change the Chinese values in the Layout to English
l_dict = {
    'LAV':'Unknown',
    '':'Unknown',
    '1房2廳':'1Room-2LivingRooms',
    '1房(1套)2廳,開放廚':'1Room(1Suite)-2LivingRooms-1OpenKitchen',
    '1房2廳,開放廚':'1Room(1Suite)-2LivingRooms-1OpenKitchen',
    '2房2廳,開放廚':'2Rooms-2LivingRooms-1OpenKitchen',
    '4房(1套)2廳1工':"4Rooms(1Suite)-2LivingRooms-1Maid'sRoom",
    '3房(1套)2廳1貯':"3Rooms(1Suite)-2LivingRooms-1StorageRoom",
    '2房2廳':'2Rooms-2LivingRooms',
    '開放式,開放廚':'Studio-1OpenKitchen',
    '3房2廳':'3Rooms-2LivingRooms',
    '2房2廳1貯':'2Rooms-2LivingRooms-1StorageRoom',
    '2房(1套)2廳1工1貯':"2Rooms(1Suite)-2LivingRooms-1Maid'sRoom-1StorageRoom",
    '3房(1套)2廳':'3Rooms(1Suite)-2LivingRooms',
    '3房(1套)2廳,開放廚':'3Rooms(1Suite)-2LivingRooms-1OpenKitchen',
    '3房(1套)2廳1工':"3Rooms(1Suite)-2LivingRooms-1Maid'sRoom"
}

data = data.replace({'Layout':l_dict})

In [13]:
#convert the Input Date and OP Date values to be in date format
for i in data.index:
    if data['Input Date'][i] =='':
        pass
    else:
        data['Input Date'][i] = pd.to_datetime(data['Input Date'][i])

    if data['OP Date'][i] =='':
        pass
    else:
        data['OP Date'][i] = pd.to_datetime(data['OP Date'][i], format='%Y')

/Users/Lawrence/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/Lawrence/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [14]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

In [15]:
data.head(3)

,Input Date,Address,Floor,Unit,Size(ft²),Price,Price/ft²,District,Street,Building,Usage,OP Date,Facing,Layout
0,2021-06-25 00:00:00,"FLOOR 5, CANTON RD 1037",5,NaN,NaN,20000000.0,NaN,Mong Kok,CANTON RD 1037,NaN,Residential,,Uknown,Unknown
1,2021-06-25 00:00:00,"FLOOR 4, CANTON RD 1037",4,NaN,NaN,20000000.0,NaN,Mong Kok,CANTON RD 1037,NaN,Residential,,Uknown,Unknown
2,2021-06-25 00:00:00,"FLOOR 3, CANTON RD 1037",3,NaN,NaN,20000000.0,NaN,Mong Kok,CANTON RD 1037,NaN,Residential,,Uknown,Unknown


In [16]:
#add new columns to the dataframe for room types
new_cols = ['Room','Suite','LivingRoom','OpenKitchen','Studio','StorageRoom',"Maid'sRoom"]

for c in new_cols:
    data[c] = 0

In [17]:
#extract the number of different room types
roomtype = new_cols[1:]

for i in data.index:
    if data['Layout'][i] == 'Unknown':
        continue
  
    if data['Layout'][i][0] =='S':
        pass
    else: 
        data['Room'][i] = pd.to_numeric(data['Layout'][i][0])
    
    for room in roomtype:
        if room in data['Layout'][i]:
            if room == 'Studio':
                data[room][i] = 1
            else:
                data[room][i] = pd.to_numeric(data['Layout'][i][data['Layout'][i].find(room)-1])
        else:
            pass
    

/Users/Lawrence/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/Lawrence/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [18]:
#calcuate the age of the building
data['Age_of_Building'] = 0

for i in data.index:
    if data['OP Date'][i] == '':
        pass
    else:
        data['Age_of_Building'][i] = relativedelta(datetime.now(), data['OP Date'][i]).years

/Users/Lawrence/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [20]:
data.head(20)

,Input Date,Address,Floor,Unit,Size(ft²),Price,Price/ft²,District,Street,Building,Usage,OP Date,Facing,Layout,Room,Suite,LivingRoom,OpenKitchen,Studio,StorageRoom,Maid'sRoom,Age_of_Building
0,2021-06-25 00:00:00,"FLOOR 5, CANTON RD 1037",5,NaN,NaN,20000000.0,NaN,Mong Kok,CANTON RD 1037,NaN,Residential,,Uknown,Unknown,0,0,0,0,0,0,0,0
1,2021-06-25 00:00:00,"FLOOR 4, CANTON RD 1037",4,NaN,NaN,20000000.0,NaN,Mong Kok,CANTON RD 1037,NaN,Residential,,Uknown,Unknown,0,0,0,0,0,0,0,0
2,2021-06-25 00:00:00,"FLOOR 3, CANTON RD 1037",3,NaN,NaN,20000000.0,NaN,Mong Kok,CANTON RD 1037,NaN,Residential,,Uknown,Unknown,0,0,0,0,0,0,0,0
3,2021-06-25 00:00:00,"FLOOR 2, CANTON RD 1037",2,NaN,NaN,20000000.0,NaN,Mong Kok,CANTON RD 1037,NaN,Residential,,Uknown,Unknown,0,0,0,0,0,0,0,0
4,2021-06-25 00:00:00,"UNIT N8, FLOOR 5, W LUXE, ON YIU ST 5",5,N8,313.0,5800000.0,18530.0,Shatin,ON YIU ST 5,W LUXE,Office,2020-01-01 00:00:00,Northeastern,Unknown,0,0,0,0,0,0,0,1
5,2021-06-25 00:00:00,"UNIT R532, FLOOR CP, FESTIVAL CITY PH 03, MEI ...",CP,R532,NaN,2530000.0,NaN,Shatin,MEI TIN RD 1,FESTIVAL CITY PH 03,Car Park,2011-01-01 00:00:00,Uknown,Unknown,0,0,0,0,0,0,0,10
6,2021-06-25 00:00:00,"UNIT 234, FLOOR B2, KINGSWOOD PH 01 LOCWOOD CT...",B2,234,NaN,1286000.0,NaN,Tin Shui Wai,TIN WU RD 1,KINGSWOOD PH 01 LOCWOOD CT,Car Park,1992-01-01 00:00:00,Uknown,Unknown,0,0,0,0,0,0,0,29
7,2021-06-25 00:00:00,FLOOR 2,2,NaN,700.0,7280000.0,10400.0,Tsuen Wan,NaN,NaN,Village,,Uknown,Unknown,0,0,0,0,0,0,0,0
8,2021-06-25 00:00:00,"UNIT B, FLOOR 7, CRESCENT GREEN TWR 08, KO PO ...",7,B,1006.0,13033000.0,12955.0,Yuen Long,KO PO PATH 3,CRESCENT GREEN TWR 08,Residential,2020-01-01 00:00:00,East,3Rooms(1Suite)-2LivingRooms-1Maid'sRoom,3,1,2,0,0,0,1,1
9,2021-06-25 00:00:00,"FLOOR G, HONG MEI TSUEN 13A",G,NaN,700.0,5800000.0,8286.0,Yuen Long,HONG MEI TSUEN 13A,NaN,Village,2011-01-01 00:00:00,Uknown,Unknown,0,0,0,0,0,0,0,10
